# Data Science Salaries - Part 2: Error Analysis and Improvements

## Error Analysis Conclusions & Work Plan

Based on the error analysis from Part 1, we identified several key issues:

- **Salary Range Bias**:
   - Model significantly underestimates high-salary positions (>250k USD)
   - Negative skew in error distribution

- **Feature Importance Issues**:
   - Employee residence dominates with ~0.40 importance score
   - Work year and remote ratio show very low importance (<0.05)
   - Potential sparsity issues with job titles

- **Experience Level Patterns**:
   - Largest error variance in Executive (EX) level
   - Significant outliers in Senior (SE) level

Loading the data & model from Part 1

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, RobustScaler, QuantileTransformer, LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from category_encoders import TargetEncoder
from imblearn.over_sampling import SMOTE
import xgboost as xgb

# Set style for better visualizations
sns.set_style("whitegrid")
sns.set_palette("husl")

# Load the original data and model
path = "data\\ds_salaries.csv"
df = pd.read_csv(path)

# Import the prepare_data function from part1
from part1 import prepare_data, build_model
processed_data = prepare_data(df)
model, X_test, y_test, y_pred, X_train, feature_cols = build_model(processed_data)

### Root Cause Analysis

Let's analyze potential causes for the observed issues:

#### Data Distribution Problems (Data Imbalance)

In [ ]:
# # Analyze salary distribution
# plt.figure(figsize=(10, 6))
# sns.histplot(data=processed_data, x='salary_in_usd', bins=50)
# plt.title('Salary Distribution')
# plt.xlabel('Salary (thousands USD)')
# plt.ylabel('Count')
# plt.show()

# Print summary statistics
print("Salary Distribution Statistics:")
print(processed_data['salary_in_usd'].describe())

# Calculate skewness
print(f"\nSkewness: {processed_data['salary_in_usd'].skew():.2f}")

# Analyze salary distribution
plt.figure(figsize=(15, 5))
  
# Original distribution
plt.subplot(1, 3, 1)
sns.histplot(processed_data['salary_in_usd'], bins=30)
plt.title('Original Salary Distribution')
    
# Log-transformed distribution
plt.subplot(1, 3, 2)
sns.histplot(np.log1p(processed_data['salary_in_usd']), bins=30)
plt.title('Log-Transformed Salary Distribution')
    
# QQ plot for log-normality
plt.subplot(1, 3, 3)
stats.probplot(np.log1p(processed_data['salary_in_usd']), dist="norm", plot=plt)
plt.title('Log-Normal Q-Q Plot')
    
plt.tight_layout()
plt.show()

We observe:
   - Right-skewed salary distribution
   - Underrepresentation of high-salary positions
   - Possible outliers affecting model training

**Some Potential Solutions:**
   - Custom sampling strategy based on salary ranges
   - Handle outliers
   - Consider log transformation for salary values


#### Feature Engineering Issues

In [1]:
# Analyze feature cardinality
categorical_cols = ['job_title', 'employee_residence', 'company_location']

print("Feature Cardinality Analysis:")
for col in categorical_cols:
    unique_count = processed_data[col].nunique()
    top_5_freq = processed_data[col].value_counts().head()
    print(f"\n{col}:")
    print(f"Unique values: {unique_count}")
    print("Top 5 most frequent values:")
    print(top_5_freq)

We observe:
   - High cardinality (too many unique values) in categorical variables, e.g. job title or location
   - Simple label encoding might not capture relationships
   - Missing interaction effects between features
   - Overall basic feature preprocessing


**Some Potential Solutions**
- Group similar job titles
- Implement target encoding for categorical variables (crucial for high-cardinality features like job_title)
- Create experience-title interaction features (captures career progression patterns)
- Develop remote work impact factors (especially relevant in today's job market)
- Include company size-location interactions (captures market dynamics)

In [ ]:
def improved_preprocessing(df):
    data = df.copy()
    
    # Create salary buckets to handle high-salary cases (outliers) better
    data['salary_bucket'] = pd.qcut(data['salary_in_usd'], q=5, labels=['Very Low', 'Low', 'Medium', 'High', 'Very High'])
    
    # Convert salary to thousands and log transform
    data['salary_in_usd'] = data['salary_in_usd'] / 1000
    data['log_salary'] = np.log1p(data['salary_in_usd'])
    
    # Group similar job titles
    data['job_category'] = data['job_title'].apply(lambda x: 
        'Manager' if any(term in x for term in ['Manager', 'Lead', 'Head', 'Director'])
        else 'Analyst' if any(term in x for term in ['Analytics', 'Analyst', 'Strategist', 'Consultant', 'Business', 'BI'])
        else 'Scientist' if any(term in x for term in ['Scientist', 'Research', 'Researcher', 'Learning'])
        else 'Engineer' if any(term in x for term in ['Engineer', 'Developer', 'Programmer', 'Architect'])
        else 'Other')
    
    # Remote Work Impact Factors
    df['remote_category'] = pd.cut(
        df['remote_ratio'],
        bins=[-1, 0, 50, 100],
        labels=['on_site', 'hybrid', 'remote']
    )
    # Calculate average salary by remote category
    remote_salary_avg = df.groupby('remote_category')['salary_in_usd'].transform('mean')
    df['remote_salary_ratio'] = df['salary_in_usd'] / remote_salary_avg
    # Create remote-experience interaction
    df['remote_experience_interaction'] = df['remote_ratio'] * df['experience_numeric']

    # Company Size-Location Interactions
    size_mapping = {
        'S': 1,  # Small
        'M': 2,  # Medium
        'L': 3   # Large
    }
    df['size_numeric'] = df['company_size'].map(size_mapping)    
    # Calculate location-specific size impact
    size_location_avg = df.groupby(['company_location', 'company_size'])['salary_in_usd'].transform('mean')
    df['size_location_salary_ratio'] = df['salary_in_usd'] / size_location_avg
    # Create size-remote interaction
    df['size_remote_interaction'] = df['size_numeric'] * df['remote_ratio']
    
    # Experience-Title Interaction Features
    exp_mapping = {
        'EN': 1,  # Entry level
        'MI': 2,  # Mid level
        'SE': 3,  # Senior
        'EX': 4   # Executive
    }
    df['experience_numeric'] = df['experience_level'].map(exp_mapping)    
    for i in range(5):  # Number of embedding dimensions
        df[f'exp_title_interaction_{i}'] = (
            df['experience_numeric'] * df[f'job_title_embedding_{i}']
        )
    
    # Target encoding
    te = TargetEncoder()
    categorical_cols = ['experience_level', 'employment_type', 'job_title', 'job_category', 
                       'employee_residence', 'company_location', 'company_size',
                       'experience_remote', 'salary_bucket', 'remote_category']

    for col in categorical_cols:
        data[col + '_encoded'] = te.fit_transform(data[col])
    
    return data

## Improving Model Performance

Other than the above issues, we also observe weaknesses in the initial baseline model, which we will discuss below together with potential solutions.

#### Model Limitations
   - Linear scaling not suitable for salary ranges
   - Single model approach for all salary ranges
   - Basic validation strategy
   - Limited hyperparameter optimization
   - Loss function equally weights all errors, leading to bias towards the mean

**Model & Training Possible Improvements**
   - Use quantile regression for better uncertainty estimation
   - Use k-fold cross-validation
   - Implement hyperparameter tuning
   - Add regularization techniques, e.g. L1 (Lasso), L2 (Ridge), early stopping, feature dropout, gradient clipping
   - Use weighted sampling for underrepresented cases

#### Enhanced Data Preprocessing

In [ ]:
# Apply improved preprocessing
enhanced_processed_data = improved_preprocessing(df)

# Define features for modeling
feature_cols = [col for col in enhanced_processed_data.columns 
               if col.endswith('_encoded') or 
               col.endswith('_interaction') or
               col in ['work_year', 'remote_salary_ratio', 'size_location_salary_ratio']]

X = enhanced_processed_data[feature_cols]
y = enhanced_processed_data['log_salary']

#### Weighted Sampling for Underrepresented Cases

In [ ]:
def calculate_sample_weights(df):
    # Calculate weights based on salary buckets
    salary_counts = df['salary_bucket'].value_counts()
    weights = 1 / salary_counts[df['salary_bucket']]
    return weights / weights.sum() * len(weights)

sample_weights = calculate_sample_weights(processed_data)

# Split data with stratification
X_train, X_test, y_train, y_test, weights_train, weights_test = train_test_split(
    X, y, sample_weights, test_size=0.2, random_state=42,
    stratify=processed_data['salary_bucket'])

# Use RobustScaler instead of StandardScaler to handle outliers better
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#### Hyperparameter Tuning with Cross-Validation

In [ ]:
# Define parameter grid for RandomizedSearchCV
param_grid = {
    'max_depth': [3, 4, 5, 6, 7, 8],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'n_estimators': [100, 200, 300, 400, 500],
    'min_child_weight': [1, 3, 5, 7],
    'gamma': [0, 0.1, 0.2, 0.3, 0.4],
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
    'reg_alpha': [0, 0.1, 0.5, 1.0],
    'reg_lambda': [0.1, 0.5, 1.0, 1.5, 2.0]
}

# Initialize base model with quantile regression objective
base_model = xgb.XGBRegressor(
    objective='reg:quantileerror',
    # L1 regularization (analogous to Lasso)
    alpha=0,
    # L2 regularization (analogous to Ridge)
    lambda_=1.0,
    # Prevent overfitting on individual trees
    max_depth=6,
    min_child_weight=5,
    gamma=0.1,
    # Subsample rows and columns for regularization
    subsample=0.8,
    colsample_bytree=0.8,
    # Enable early stopping
    early_stopping_rounds=10,
    # Gradient clipping
    max_delta_step=1,
    # Slower learning rate for better generalization
    learning_rate=0.01,
    n_estimators=1000,
    random_state=42
)

# Set up K-Fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Perform RandomizedSearchCV with cross-validation
random_search = RandomizedSearchCV(
    estimator=base_model,
    param_distributions=param_grid,
    n_iter=100,
    scoring='neg_mean_squared_error',
    cv=kf,
    random_state=42,
    n_jobs=-1
)

# Fit the random search
random_search.fit(
    X_train_scaled, 
    y_train
)

print("Best parameters found:")
for param, value in random_search.best_params_.items():
    print(f"{param}: {value}")

#### Training Final Model with Quantile Regression

In [ ]:
# Train models for different quantiles
quantiles = [0.1, 0.5, 0.9]
quantile_models = {}

for q in quantiles:
    model = xgb.XGBRegressor(
        objective='reg:quantileerror',
        quantile_alpha=q,
        **random_search.best_params_,
        # L1 regularization (analogous to Lasso)
        alpha=0.5,
        # L2 regularization (analogous to Ridge)
        lambda_=1.0,
        # Prevent overfitting on individual trees
        max_depth=6,
        min_child_weight=5,
        gamma=0.1,
        # Subsample rows and columns for regularization
        subsample=0.8,
        colsample_bytree=0.8,
        # Enable early stopping
        early_stopping_rounds=10,
        # Gradient clipping
        max_delta_step=1,
        # Slower learning rate for better generalization
        learning_rate=0.01,
        n_estimators=1000,
        random_state=42
    )
    
    model.fit(
        X_train_scaled, 
        y_train
    )
    
    quantile_models[q] = model

# Make predictions for each quantile
predictions = {}
for q, model in quantile_models.items():
    predictions[q] = model.predict(X_test_scaled)